In [1]:
import os
import random
import pandas as pd
from sqlalchemy import create_engine
import chromadb
from dotenv import load_dotenv

In [ ]:
def get_data(database_url):
    engine = create_engine(database_url)
    query = "SELECT * FROM reddit_usernames_comments"
    df = pd.read_sql(query, engine)
    return df

In [ ]:
database_url = os.getenv('DATABASE_URL')
df = get_data(database_url)
df.info()

In [ ]:
df.to_csv('data/comments.csv', index=False)

In [24]:
df = pd.read_csv('comments.csv')
df.head()

,username,comments
0,LoveAGoodTwist,"Female, Kentucky. 4 years out. Work equine on..."
1,wahznooski,"As a woman of reproductive age, fuck Texas|As ..."
2,Churro_The_fish_Girl,what makes you want to become a vet?|what make...
3,abarthch,"I see of course there are changing variables, ..."
4,VoodooKing,I have 412+ and faced issues because wireguard...


In [34]:
comments_dict = df.set_index('username')['cleaned_comments'].to_dict()

In [36]:
random_200_rows = {k: comments_dict[k] for k in random.sample(comments_dict.keys(), 200)}

/var/folders/4b/y66j_ccs6336dsqfhxjxlcqw0000gn/T/ipykernel_1192/1903984158.py:1: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random_200_rows = {k: comments_dict[k] for k in random.sample(comments_dict.keys(), 200)}


In [43]:
random_200_rows

In [40]:
# Divide the dictionary into two parts - 100 rows each
dict_part1 = {key: random_200_rows[key] for key in list(random_200_rows.keys())[:100]}
dict_part2 = {key: random_200_rows[key] for key in list(random_200_rows.keys())[100:]}

In [44]:
# Divide the dictionary into two parts - 100 rows each
dict_part1_1 = {key: dict_part1[key] for key in list(dict_part1.keys())[:50]}
dict_part1_2 = {key: dict_part1[key] for key in list(dict_part1.keys())[50:]}

In [49]:
# Calculate the total number of characters in the dictionary including both keys and values
total_characters = sum(len(key) + sum(len(value) for value in values) for key, values in dict_part1_2.items())

# Display the total number of characters
print("Total number of characters in the dictionary:", total_characters)


Total number of characters in the dictionary: 142313


In [55]:
load_dotenv()

True

In [56]:
from anthropic import Anthropic
client = Anthropic()
MODEL_NAME = "claude-3-opus-20240229"

In [57]:
def get_completion(client, prompt):
    return client.messages.create(
        model=MODEL_NAME,
        max_tokens=4048,
        messages=[{
            "role": 'user', "content":  prompt
        }]
    ).content[0].text

In [66]:
prompt=f"""
The following dictionary contains reddit comments where the keys are usernames and the values are the corresponding comments:
{dict_part1_2}

You are to analyze the comments and determine who made the comment.

Your classifier should run through this list and determine if they are of these categories:

Medical Doctor:
This label should only include practicing doctors and or consultants to doctors/clinics. 
Medical school students, nurses or medical professionals who aren’t doctors should go into the “Other” label instead.

Veterinarian:
This label should only include practicing vets and/or consultants to vets/clinics. 
Veterinarian students or veterinarian technicians should go into the “Other” label instead.

Other:
Anyone who does not fit within the Medical Doctor, or a Veterinarian label.

Your response should contain a Python dictionary with keys as the usernames and values as the assigned label, don't include the actual comments in your response.
"""

In [67]:
completion = get_completion(client, prompt)
print(completion)

Here is the Python dictionary classifying each user based on their comments:

{
    'avero34': 'Other',
    'Huntseatqueen': 'Other',
    'PracticalAioli6764': 'Other',
    'MSmith3813': 'Veterinarian',
    'relevancycheck': 'Other',
    'nextlevelou': 'Other',
    'PankNails': 'Other',
    'o_my_captain': 'Veterinarian',
    'Prestigious_Wheel128': 'Other',
    'Ventaura': 'Veterinarian',
    'Affectionate_Net_213': 'Veterinarian',
    'wachinayn': 'Other',
    'Altruistic_Ad4949': 'Veterinarian',
    'glitchgorge': 'Other',
    'mckenna310': 'Other',
    'PetLuxincps': 'Veterinarian',
    'big_fat_momma_llama': 'Other',
    'Zealousideal-Dog-31': 'Veterinarian',
    'Mighty_Buddha': 'Other',
    'CopiousCoffee_': 'Other',
    'kriki200': 'Other',
    'techtornado': 'Other',
    'shumbalar': 'Other',
    'Resident_Bitch': 'Other',
    'Adventurous_Pear_711': 'Other',
    'FUmods89': 'Other',
    'satsukitea': 'Other',
    'Ectophylla-alba': 'Other',
    'MyIslandHomes': 'Other',
    '

In [68]:
from dict_result import my_dict

In [69]:
len(my_dict)

189

In [70]:
# Assuming df is your parent dataframe and user_label_dict is your dictionary
# Create a DataFrame from the dictionary
user_label_df = pd.DataFrame.from_dict(my_dict, orient='index', columns=['label'])

# Reset index to make 'username' a column instead of index
user_label_df.reset_index(inplace=True)
user_label_df.rename(columns={'index': 'username'}, inplace=True)

# Merge user_label_df with the parent dataframe on the 'username' column
merged_df = pd.merge(df, user_label_df, on='username', how='inner')

# Select the desired columns from the merged dataframe
selected_df = merged_df[['username', 'comments', 'label']]


In [71]:
selected_df.head()

,username,comments,label
0,Different-Ad1735,"Hi,\n\nThis old tutorial does not work for me,...",Other
1,Most-Exit-5507,In high school I found a youtube channel calle...,Other
2,Frizzyawkward,I’m a soon to be veteran looking to get into v...,Other
3,OkTeaching4656,I currently work at Purdue animal hospital as ...,Other
4,Flashy-Ad-2261,"yes, i quit to. 0,013 myst a month is no good...",Other


In [72]:
selected_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   username  188 non-null    object
 1   comments  188 non-null    object
 2   label     188 non-null    object
dtypes: object(3)
memory usage: 4.5+ KB


In [73]:
more_doc = [
    {"username": "doc001", "comments": "In my experience, fostering a culture of mutual respect and collaboration in the clinic leads to better patient outcomes and team morale. Let's prioritize communication and teamwork for the benefit of our patients",  "label": "Medical Doctor"},
    {"username": "doc002", "comments": "As a practicing physician, I've found that humility and empathy go a long way in building strong relationships with colleagues and staff. Let's strive for a culture of inclusivity and support in our medical community.", "label": "Medical Doctor"},
    {"username": "doc003", "comments": "It's important to recognize the diverse skill sets and contributions of every member of the healthcare team. We're all here to serve our patients, and acknowledging each other's expertise is key to delivering high-quality care.", "label": "Medical Doctor"},
    {"username": "doc004", "comments": "I've learned throughout my career that effective communication and mutual respect are essential for a successful medical practice. Let's focus on building trust and collaboration among our team.", "label": "Medical Doctor" },
    {"username": "doc005", "comments": "As doctors, we must lead by example in fostering a culture of professionalism and respect in our workplace. Let's prioritize open communication and constructive feedback to promote a positive working environment.", "label": "Medical Doctor"},
    {"username": "doc006", "comments": "In my experience, addressing conflicts and misunderstandings openly and respectfully is crucial for maintaining a cohesive healthcare team. Let's encourage open dialogue and constructive problem-solving.", "label": "Medical Doctor"},
    {"username": "doc007", "comments": "As a physician, I believe that creating a supportive and inclusive work environment is essential for the well-being of both patients and staff. Let's commit to treating each other with kindness and respect.",  "label": "Medical Doctor"},
    {"username": "doc008", "comments": "I've found that investing in mentorship and professional development opportunities for staff can greatly enhance team cohesion and morale. Let's support each other's growth and success in our medical careers.", "label": "Medical Doctor"},
    {"username": "doc009", "comments": "As doctors, we have a responsibility to advocate for a workplace culture that values diversity, equity, and inclusion. Let's work together to create a welcoming and supportive environment for all members of our team.", "label": "Medical Doctor"},
    {"username": "doc009", "comments": "In my practice, I've seen firsthand the positive impact of fostering a culture of trust and collaboration among healthcare professionals. Let's strive to build strong relationships and mutual respect in our workplace.", "label": "Medical Doctor"}
]

In [75]:
# Convert the list of dictionaries into a DataFrame
more_doc_df = pd.DataFrame(more_doc)

# Concatenate the new rows DataFrame with the original DataFrame
selected_df = pd.concat([selected_df, more_doc_df], ignore_index=True)

In [76]:
selected_df

,username,comments,label
0,Different-Ad1735,"Hi,\n\nThis old tutorial does not work for me,...",Other
1,Most-Exit-5507,In high school I found a youtube channel calle...,Other
2,Frizzyawkward,I’m a soon to be veteran looking to get into v...,Other
3,OkTeaching4656,I currently work at Purdue animal hospital as ...,Other
4,Flashy-Ad-2261,"yes, i quit to. 0,013 myst a month is no good...",Other
...,...,...,...
193,doc006,"In my experience, addressing conflicts and mis...",Medical Doctor
194,doc007,"As a physician, I believe that creating a supp...",Medical Doctor
195,doc008,I've found that investing in mentorship and pr...,Medical Doctor
196,doc009,"As doctors, we have a responsibility to advoca...",Medical Doctor


In [77]:

selected_df['label'] = selected_df['label'].replace({'Veterinarian': 'Vet', 'Medical Doctor': 'Med_Doc'})


In [78]:
selected_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   username  198 non-null    object
 1   comments  198 non-null    object
 2   label     198 non-null    object
dtypes: object(3)
memory usage: 4.8+ KB


In [79]:
selected_df['label'].value_counts()

label
Other      131
Vet         46
Med_Doc     21
Name: count, dtype: int64

In [80]:
selected_df.to_csv('selected.csv', index=False)